In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import Model, layers
from tensorflow.keras.datasets import mnist

In [2]:
num_classes = 10 # total classes (0-9 digits).
num_features = 784 # data features (img shape: 28*28).

learning_rate = 0.1
training_steps = 2000
batch_size = 256
display_step = 100

n_hidden_1 = 128 
n_hidden_2 = 256 

In [3]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train, x_test = np.array(x_train, np.float32), np.array(x_test, np.float32)
x_train, x_test = x_train.reshape([-1, num_features]), x_test.reshape([-1, num_features])
x_train, x_test = x_train / 255., x_test / 255.

In [4]:
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.repeat().shuffle(5000).batch(batch_size).prefetch(1)

In [16]:
class NeuralNet(Model):
    def __init__(self):
        super(NeuralNet, self).__init__()
        self.fc1 = layers.Dense(n_hidden_1, activation=tf.nn.relu)
        self.fc2 = layers.Dense(n_hidden_2, activation=tf.nn.relu)
        self.out = layers.Dense(num_classes, activation=tf.nn.softmax)
        
    def call(self, x, is_training=False):
        x = self.fc1(x)
        x = self.out(x)
        if not is_training:
            x = tf.nn.softmax(x)
        return x

model = NeuralNet()

In [22]:
def cross_entropy_loss(x, y):
    y = tf.cast(y, tf.int64)
    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=x)
    return tf.reduce_mean(loss)

def accuracy(y_pred, y_true):
    correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.cast(y_true, tf.int64))
    return tf.reduce_mean(tf.cast(correct_prediction, tf.float32), axis=-1)

In [7]:
optimizer = tf.optimizers.SGD(learning_rate)

In [19]:
def run_optimization(x,y):
    with tf.GradientTape() as g:
        pred = model(x, is_training=True)
        loss = cross_entropy_loss(pred, y)
        
    gradients = g.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

In [23]:
for step, (batch_x, batch_y) in enumerate(train_data.take(training_steps), 1):
    run_optimization(batch_x, batch_y)
    
    if step % display_step == 0:
        pred = model(batch_x, is_training=True)
        loss = cross_entropy_loss(pred, batch_y)
        acc = accuracy(pred, batch_y)
        print("step: %i, loss: %f, accuracy: %f" % (step, loss, acc))

step: 100, loss: 1.861001, accuracy: 0.667969
step: 200, loss: 1.867190, accuracy: 0.621094
step: 300, loss: 1.749029, accuracy: 0.757812
step: 400, loss: 1.759238, accuracy: 0.730469
step: 500, loss: 1.725115, accuracy: 0.777344
step: 600, loss: 1.660356, accuracy: 0.871094
step: 700, loss: 1.605304, accuracy: 0.894531
step: 800, loss: 1.609697, accuracy: 0.886719
step: 900, loss: 1.629842, accuracy: 0.863281
step: 1000, loss: 1.582073, accuracy: 0.929688
step: 1100, loss: 1.581905, accuracy: 0.914062
step: 1200, loss: 1.573627, accuracy: 0.914062
step: 1300, loss: 1.590415, accuracy: 0.890625
step: 1400, loss: 1.550007, accuracy: 0.941406
step: 1500, loss: 1.561934, accuracy: 0.914062
step: 1600, loss: 1.579360, accuracy: 0.906250
step: 1700, loss: 1.559459, accuracy: 0.925781
step: 1800, loss: 1.546293, accuracy: 0.941406
step: 1900, loss: 1.544258, accuracy: 0.941406
step: 2000, loss: 1.585490, accuracy: 0.894531
